# First Assignment - FINTECH 540 - Machine Learning for FinTech

In this assignment, you will gain hands-on experience applying linear models to financial market data. Specifically, you will work with time series prices of the 30 constituents of the *Dow Jones Industrial Average (DJIA)* Index. The dataset covers the period from June $2^{nd}$, 2017, through June $2^{nd}$, 2023. The price series of the ETF associated with the DJIA index is also provided, whose symbol is *DIA*. The dataset is uploaded on Sakai in the same place where you found this notebook.

You will deal with three consecutive tasks, so in general, you can only perform a task if you have solved the previous one. You can obtain at most 100 points for this home assignment. The tasks are briefly summarized below, and you can find the relative prompt in each subsection of this notebook:
- Build descriptive linear models (CAPM) for all the index constituents (*20 points*).
- Select a subset of constituents and fit a predictive linear model to forecast the index value (*40 points*).
- Repeat the linear modeling exercise using boostrapped returns (*40 points*).

## About this notebook

You only need to write the final code between the `### START CODE HERE ###` and `### END CODE HERE ###` comments. You can create more cells to experiment with and prepare your final code at your convenience. Remember to put the final version of the code where it is asked. Before submitting, remember to fully run your notebook from the start to the end to ensure that there will be no runtime error. Avoiding following such guidelines will result in a decrease in the total points.

## Task 1 - Build descriptive linear models (CAPM) for all the index constituents (*20 points*)

The Capital Asset Pricing Model (CAPM) is represented as:

$$R_i - R_f =   \beta_i (R_m - R_f) + e_i$$

Where:
- $R_i$ is the return of the asset or security $i$.
- $R_f$ is the risk-free rate, representing the return on a risk-free investment.
- $\beta_i$ is the beta of the asset $i$, which measures its sensitivity to market movements.
- $R_m$ is the market portfolio's return (the index).
- $e_i$ is the error term or residual representing unexplained variation in the asset's return.

The CAPM equation helps estimate the return of an asset based on its risk relative to the market and the risk-free rate. You can calculate the daily risk-free rate by using the following formula.

$$ r_{\text{daily}} = \left(1 + r_{\text{annual}}\right)^{\frac{1}{365}} - 1 $$

Where:
- $r_{\text{daily}}$ is the daily yield. It represents the expected daily return on investment.
- $ r_{\text{annual}} $ is the annual yield. It represents the expected annual return on investment.
- The formula assumes daily compounding, meaning the investment's return is calculated daily over a year (365 days). It allows to do the modeling based on daily returns.

For this task, you can use an annual yield of *5.482%* per the annualized U.S. 3-month Treasury Bill yield.

To solve this part of the homework, you have to:
- Compute the daily yield from the annualized provided in the prompt.
- Prepared the data to fit the CAPM for each company in the DJIA index described above.
- Fit the CAPM for each company and check the estimated sensitivity to market movements.
- Select a subset of stocks sensitive to market movements between 0.85 and 1.15. Before including a symbol, ensure the estimated sensitivity is statistically significant. Store the symbols in a Python list before moving to the next task.

Before performing the CAPM modeling, remember to split the dataset into a training set and a test set and use only the training set to perform Task 1. Use *2022-01-01* as a cutoff date. Ensure the cutoff date is included in the test set and not in the train set.

**Motivation behind the task**

Fitting individual CAPM models allows for a detailed assessment of each stock's risk profile. CAPM provides a systematic way to quantify the sensitivity of each stock's returns to market movements, as measured by the beta coefficient. This individual assessment is valuable because different stocks may exhibit varying levels of market sensitivity.

Selecting stocks based on their beta values is usually a risk-based approach to portfolio construction. By choosing stocks with higher (lower) beta values, you are essentially selecting those that tend to exhibit greater (lower) price volatility in response to market fluctuations. This can be seen as a deliberate strategy to include riskier (safer) assets in the portfolio.

This task will set the basis for selecting a subset of index constituents to be used for a predictive model. 

**Grading Criteria**

- **Data Preparation (10 points)**: Points will be awarded for preparing the data appropriately for the modeling task.

- **CAPM Model Fitting (10 points)**: Points will be awarded based on the correctness and completeness of the CAPM models, including accurate significance evaluation and the subset of stock selection based on the beta estimations.

In [5]:
### START CODE HERE ###
# 导入必要的库
import pandas as pd
import statsmodels.api as sm

# 计算每日无风险利率
annual_yield = 5.482 / 100  # 将年收益率转为小数
daily_yield = (1 + annual_yield) ** (1/365) - 1  # 计算每日收益率

# 载入道琼斯成分股数据
csv_file_path = 'dows_daily.csv'
data = pd.read_csv(csv_file_path)

# 将 'Date' 列转换为 datetime 类型
data['Date'] = pd.to_datetime(data['Date'])

# 划分训练集和测试集（以 2022-01-01 为分割点）
train_data = data[data['Date'] < '2022-01-01']
test_data = data[data['Date'] >= '2022-01-01']

# 计算每只股票的日收益率
stocks = data.columns[1:]  # 排除 'Date' 列
train_returns = train_data.set_index('Date')[stocks].pct_change().dropna()  # 训练集日收益率
test_returns = test_data.set_index('Date')[stocks].pct_change().dropna()  # 测试集日收益率

# 计算市场（DIA）的日收益率
train_market_return = train_data.set_index('Date')['DIA'].pct_change().dropna()  # 训练集市场回报
test_market_return = test_data.set_index('Date')['DIA'].pct_change().dropna()  # 测试集市场回报

# 输出训练集日收益率的前几行，检查数据是否正确
print(train_returns.head())
print(train_market_return.head())

# 定义一个函数，用于计算每只股票的CAPM Beta值
def calculate_capm_beta(stock_returns, market_returns, risk_free_rate):
    # 计算超额回报（从回报中减去无风险利率）
    excess_stock_returns = stock_returns - risk_free_rate
    excess_market_returns = market_returns - risk_free_rate

    # 给市场回报加上常数项（用于计算截距）
    X = sm.add_constant(excess_market_returns)
    y = excess_stock_returns

    # 拟合回归模型
    model = sm.OLS(y, X).fit()

    # 提取Beta系数和p值
    beta = model.params[1]
    p_value = model.pvalues[1]

    return beta, p_value

# 用于存储符合条件的股票符号
selected_symbols = []

# 对每只股票计算CAPM Beta，并检查其显著性
for stock in train_returns.columns:
    beta, p_value = calculate_capm_beta(train_returns[stock], train_market_return, daily_yield)

    # 选择Beta值在0.85到1.15之间且p值小于0.05（具有统计显著性）的股票
    if 0.85 <= beta <= 1.15 and p_value < 0.05:
        selected_symbols.append(stock)

# 输出符合条件的股票符号
print("符合条件的股票符号:", selected_symbols)



### END CODE HERE ###

                 DIA      GS.N     NKE.N   CSCO.OQ     JPM.N     DIS.N  \
Date                                                                     
2017-06-05 -0.000236  0.003188  0.000566 -0.006879  0.001815 -0.006158   
2017-06-06 -0.002313  0.002523 -0.009998 -0.006297  0.002053 -0.009576   
2017-06-07  0.001656  0.005827  0.014291  0.001584  0.011451  0.003981   
2017-06-08  0.000661  0.013810 -0.000564  0.000000  0.012394 -0.015106   
2017-06-09  0.003823  0.016822  0.004887 -0.007593  0.023661  0.012462   

             INTC.OQ     MRK.N     CVX.N     AXP.N  ...      PG.N     IBM.N  \
Date                                                ...                       
2017-06-05  0.000551 -0.006110  0.000776  0.006115  ...  0.001693  0.002368   
2017-06-06 -0.005779 -0.006147  0.009497 -0.001520  ...  0.000676 -0.000262   
2017-06-07  0.003598 -0.009433 -0.003840  0.012175  ... -0.000338 -0.009123   
2017-06-08  0.006067 -0.013425  0.002216  0.001754  ... -0.010364  0.007418   
2017-06

C:\Users\98064\AppData\Local\Temp\ipykernel_9916\530807163.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]
C:\Users\98064\AppData\Local\Temp\ipykernel_9916\530807163.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = model.pvalues[1]
C:\Users\98064\AppData\Local\Temp\ipykernel_9916\530807163.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]
C:\Users\98064\AppData\Loca

## Task 2 - Select a subset of constituents and fit a predictive linear model to forecast the index value (*40 points*)

In this task, you will apply linear predictive modeling techniques to forecast the value of the DIA ETF on the DJIA index using the subset of its constituents you selected in the previous task. The goal is to build a predictive linear model that accurately estimates the future index return based on the historical data of selected constituent stocks. Note that to perform this predictive task, you have to prepare the data accordingly. Don't use the excess returns with respect to a daily risk-free rate for this task, but use the plain returns instead.

The predictive linear regression equation to estimate the dependent variable \(Y\) at time \(t+1\) is represented as:

$$ Y_{t+1} = \beta_0 + \beta_1 X_{1,t} + \beta_2 X_{2,t} + \ldots + \beta_k X_{k,t} + \varepsilon_{t} $$

In this equation:

- $Y_{t+1}$ represents the dependent variable at time $t+1$ that we want to predict. Note that the dependent variable is real-valued.
- $\beta_0$ is the intercept or constant term.
- $\beta_1, \beta_2, \ldots, \beta_k$ are the $k$ coefficients for the independent variables $ X_{1,t}, X_{2,t}, \ldots, X_{k,t} $ at time $t$. you can assume $k$ to be the number of selected stocks from the previous task. Note that the regressors are real-valued.
- $\varepsilon_{t}$ represents the error term at time $t$, capturing unexplained variation or noise in the dependent variable at that specific time.

Before performing the linear regression modeling, remember to split the dataset into a training set and a test set. Use *2022-01-01* as a cutoff date, the same way you did in the previous task. Make sure the cutoff date will be included in the test set and not in the train set.

Assess the performance of your predictive model using an appropriate evaluation metric for a regression problem like this one. Evaluate the model on the test set to ensure its predictive accuracy out-of-sample.

**Grading Criteria**

- **Data Preparation (15 points)**: Points will be awarded for preparing the data appropriately for the modeling task.

- **Predictive Regression Model Building (20 points)**: Points will be awarded based on the correctness and completeness of the regression model built using selected stocks' returns and the index return.

- **Model Evaluation (5 points)**: Points will be awarded based on the proper choice of evaluation metric.

In [11]:
### START CODE HERE ###
# 导入必要的库
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# 选择Task 1中筛选出的符合条件的股票符号
selected_symbols = ['DIA', 'NKE.N', 'CSCO.OQ', 'DIS.N', 'INTC.OQ', 'HD.N',
                    'UNH.N', 'MSFT.OQ', 'HON.OQ', 'CRM.N', 'IBM.N', 'MMM.N',
                    'AAPL.OQ', 'CAT.N', 'V.N', 'TRV.N']

# 获取训练集和测试集的数据
train_selected_returns = train_returns[selected_symbols]
test_selected_returns = test_returns[selected_symbols]

# 将市场回报（DIA）作为目标变量
train_target = train_market_return
test_target = test_market_return

# 将训练集和测试集的数据分别划分为特征矩阵和目标向量
X_train = train_selected_returns
y_train = train_target
X_test = test_selected_returns
y_test = test_target

# 初始化线性回归模型
model = LinearRegression()

# 训练模型
model.fit(X_train, y_train)

# 使用测试集进行预测
y_pred = model.predict(X_test)

# 计算均方误差（MSE）作为评估指标
mse = mean_squared_error(y_test, y_pred)

# 输出模型的均方误差
print(f"模型的均方误差 (MSE): {mse}")

# 输出模型的系数（每个股票的回归系数）
print("回归系数：", model.coef_)

# 输出模型的截距
print("回归截距：", model.intercept_)



### END CODE HERE ###

模型的均方误差 (MSE): 2.889457871082582e-34
回归系数： [ 1.00000000e+00 -9.20538532e-17 -6.63374547e-17 -1.05883015e-16
 -2.49927403e-16 -7.34040691e-17 -1.34128701e-18 -1.99414262e-16
 -7.69535028e-17 -8.59567856e-17 -6.44923434e-17 -1.24555935e-17
 -1.63297947e-16 -4.82109499e-17 -7.76011338e-17 -6.82594511e-18]
回归截距： 8.673617379884035e-19


## Task 3 - Augment the Dataset with Bootstrapped Alphas and Fit again the Linear Predictive Models (40 points)

In this task, we explore the concept of bootstrapped alphas and their role in predictive modeling. Bootstrapped alphas are used as proxy trading signals for real alphas that can be practically obtained. These signals are correlated with future returns and can play the role of good predictors in the predictive modeling process. Don't use the excess returns with respect to a daily risk-free rate for this task, but use the plain returns instead when you have to calculate the boostrapped alphas.

We define bootstrapped alphas $\alpha_t$ as per the formula below:

$$\alpha_{i,t} := \rho_{\text{boot}} r_{i,t+1} + \sqrt{1 - \rho_{\text{boot}}^{2}} z_{i,t}$$

where:
- $r_{i,t+1}$ represents the next period return of the traded security $i$, which is given to you.
- $z_{i,t} \sim \mathbb{N}(0,\sigma^{2})$ is a randomly drawn scalar associated for each company $i$, which is not given and you have to sample. When sampling, ensure that each sampled vector is independent of the other since you have to draw samples for each company you will use as regressors. The number of companies stays the same that you used in the previous task and that you have selected by fitting the CAPM model in task 1.
- $\sigma^{2}_{i}$ is an estimate of the true conditional variance of the security $i$, which you have to calculate based on the given returns. Note that you have to calculate those variances on the train set only. Use the same cutoff applied in the previous task to define what the training set is.
- $\rho_{\text{boot}} \in [-1,1]$ is a correlation coefficient, which you have to set equal to 0.25.

In this setting, the parameter $\rho_{\text{boot}}$ artificially regulates the strength of the trading signal you create. We remark that regressing the bootstrapped alpha $\alpha_t$ on the future returns $r_{t+1}$ results in an $R^2$ equal to $\rho^2$.

The equation above formalizes the calculation of the boostrapped alpha for a single security while you will have more than one security. Try to make your calculations as efficient as possible by computing them simultaneously. It is possible by using calculations between pandas dataframe. Remember that $z_{i,t} \sim \mathcal{N}(0,\sigma^{2}_{i})$ can be calculated as $z_{i,t} = \sqrt{\sigma^{2}_{i}}u_{i,t}$ where $u_{i,t} \sim \mathcal{N}(0,1)$. 

Once you calculate the boostrapped alphas, repeat the linear predictive forecasting exercise as in the previous task. This time you will use the boostrapped alphas as predictors, while you will keep the same target as before, the index returns. In other words, the target stays the same as in the previous task (future returns for DIA) by looking at the equation below. Still, the predictors change from the current returns of the constituents to the alpha bootstrap you have calculated.

$$ Y_{t+1} = \beta_0 + \beta_1 X_{1,t} + \beta_2 X_{2,t} + \ldots + \beta_k X_{k,t} + \varepsilon_{t} $$

To ensure reproducibility, please set the random seed to 42. Don't use another seed, and remember to set it. Avoiding to follow these guidelines will result in point deductions.

**Motivation behind the task**

In the dynamic and complex world of financial markets, predictive modeling is a potent tool to decipher underlying patterns and trends that govern security prices. Coming up with good predictors for a certain set of assets is a complicated task that is not necessarily the purpose of this assignment. The concept of bootstrapped alphas, as delineated in this exercise, emerges as a sophisticated method to engineer artificial trading signals that can potentially enhance the predictive power of financial models. It is equivalent to assuming that we have a way to predict the future returns of the index constituents. Look at the alpha bootstrap equation to understand why we are talking about future returns by looking at what the prices indicate.

The utilization of bootstrapped alphas is grounded in the mathematical formulation provided, where the alpha ($\alpha_{i,t}$) for a security $i$ at time $t$ is constructed using a combination of the next period return of the security ($r_{i,t+1}$) and a stochastic component ($z_{i,t}$) drawn from a normal distribution. This formulation allows for the incorporation of both deterministic and random elements, thereby mimicking the inherent uncertainty and volatility observed in financial markets.

By setting the correlation coefficient ($\rho_{\text{boot}}$) to 0.25, we are essentially moderating the influence of the artificial trading signal, ensuring that it does not overwhelmingly dictate the behavior of the bootstrapped alphas. This parameter, therefore, serves as a tuning knob, allowing us to control the strength of the trading signal and, consequently, its predictive power. However, you have to keep this parameter fixed for this exercise, as indicated by the prompt.

The subsequent step of employing these bootstrapped alphas as predictors in a linear predictive forecasting model is an exercise to highlight how well one can expect to forecast index returns, given a good way to predict future returns for the constituents. By replacing the current returns of the constituents with the calculated bootstrapped alphas, we are essentially enhancing the model with artificially generated yet statistically grounded signals that can potentially unveil deeper insights into the market dynamics.

**Grading Criteria**

- **Data Preparation (30 points)**: Points will be awarded for preparing the data appropriately for the modeling task.

- **Predictive Regression Model Building (5 points)**: Points will be awarded based on the correctness and completeness of the regression model built using selected stocks' boostrapped alpha and the index return.
- **Model Evaluation (5 points)**: Points will be awarded based on the proper choice of evaluation metric.

In [12]:
### START CODE HERE ###
# 导入必要的库
import numpy as np

# 定义一个函数来生成自举回报
def bootstrap_returns(returns, n_iterations=1000, sample_size=None):
    if sample_size is None:
        sample_size = len(returns)

    bootstrapped_returns = []

    # 执行自举抽样
    for _ in range(n_iterations):
        bootstrap_sample = np.random.choice(returns, size=sample_size, replace=True)
        bootstrapped_returns.append(bootstrap_sample)

    return np.array(bootstrapped_returns)

# 生成自举回报
n_iterations = 1000
bootstrapped_selected_returns = {}
for stock in selected_symbols:
    bootstrapped_selected_returns[stock] = bootstrap_returns(train_returns[stock], n_iterations)

# 将自举回报的平均值作为新的特征
bootstrapped_selected_returns_mean = {
    stock: np.mean(bootstrapped_selected_returns[stock], axis=0)
    for stock in selected_symbols
}

# 将生成的自举回报替换原始回报，作为特征进行回归建模
X_train_bootstrap = pd.DataFrame(bootstrapped_selected_returns_mean)
X_test_bootstrap = pd.DataFrame({
    stock: np.mean(bootstrap_returns(test_returns[stock], n_iterations), axis=0)
    for stock in selected_symbols
})

# 初始化线性回归模型
model_bootstrap = LinearRegression()

# 使用自举回报训练回归模型
model_bootstrap.fit(X_train_bootstrap, y_train)

# 使用测试集进行预测
y_pred_bootstrap = model_bootstrap.predict(X_test_bootstrap)

# 计算均方误差（MSE）作为评估指标
mse_bootstrap = mean_squared_error(y_test, y_pred_bootstrap)

# 输出模型的均方误差
print(f"使用自举回报的模型均方误差 (MSE): {mse_bootstrap}")

# 输出模型的回归系数
print("回归系数：", model_bootstrap.coef_)

# 输出模型的截距
print("回归截距：", model_bootstrap.intercept_)



### END CODE HERE ###

使用自举回报的模型均方误差 (MSE): 0.00013626083987485762
回归系数： [ 0.5527084  -0.34627469  0.89603472  1.10256645  0.13693596  0.0264102
  0.05317477  0.61891409 -0.67066139 -0.70575065 -0.29668262 -1.18008835
  0.72061623 -0.05256131  1.0867826   1.17013488]
回归截距： -0.002779237199441092
